<a href="https://colab.research.google.com/github/taimoortiger/Heart-Disease-Detection-using-CNN/blob/main/Heart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder ,MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.models import load_model
from keras_radam import RAdam

In [ ]:
df=pd.read_csv('heart (2).csv')
df

,Age,Sex,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,130,283,0,ST,98,N,0.0,Up,0
3,48,F,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,130,236,0,LVH,174,N,0.0,Flat,1


In [ ]:
df.isnull().sum()

Age               0
Sex               0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [ ]:
X=df.iloc[:,:-1]
y=df['HeartDisease']

#Encode using
encode = LabelEncoder()

X['Sex'] = encode.fit_transform(X['Sex'])
X['RestingECG'] = encode.fit_transform(X['RestingECG'])
X['ExerciseAngina'] = encode.fit_transform(X['ExerciseAngina'])
X['ST_Slope'] = encode.fit_transform(X['ST_Slope'])

#Scaling using StandardScaler
scale = StandardScaler()
X_scaled = scale.fit_transform(X)
X_scaled

#Splitting data into training testing set
X_train, X_test, y_train, y_test=train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_scaled.shape

(918, 10)

In [ ]:
#Creating a sential model
model = Sequential()

#Add a dense layer with 128 units and relu activation funstion as input layer
model.add(Dense(130, activation='relu', input_shape=(10,)))

#Add another dense layer with 64 units and relu activation function
model.add(Dense(100, activation='relu'))

#Add the output layer as number of classes of labels with siftmax activation function
model.add(Dense(2, activation='softmax'))

#Compile the model with 'categorical_crossentropy' loss function and 'adam' optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Print the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 130)               1430      
                                                                 
 dense_4 (Dense)             (None, 100)               13100     
                                                                 
 dense_5 (Dense)             (None, 2)                 202       
                                                                 
Total params: 14732 (57.55 KB)
Trainable params: 14732 (57.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#Number of trainable parameters in 1st dense layer
layer1=10*130+130

#Number of trainable parameters in 2nd dense layer
layer2=130*100+100

#Number of trainable parameters in 3ed dense layer
layer3=100*2+2

#Total Trainable Parameters in the Model
total=layer1+layer2+layer3
print('Total Trainable Parameters in the Model = ', total)

Total Trainable Parameters in the Model =  14732


In [ ]:
# Convert target variables to one-hot encoded format for multi-class classification
from keras.utils import to_categorical
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

#Training model on the train set
dl_model=model.fit(X_train, y_train_encoded, batch_size=16, epochs=27 ,validation_split=0.2)

Epoch 1/27
37/37 [==============================] - 1s 7ms/step - loss: 0.5018 - accuracy: 0.8126 - val_loss: 0.4159 - val_accuracy: 0.8435
Epoch 2/27
37/37 [==============================] - 0s 2ms/step - loss: 0.3615 - accuracy: 0.8620 - val_loss: 0.3888 - val_accuracy: 0.8435
Epoch 3/27
37/37 [==============================] - 0s 2ms/step - loss: 0.3419 - accuracy: 0.8688 - val_loss: 0.3820 - val_accuracy: 0.8571
Epoch 4/27
37/37 [==============================] - 0s 2ms/step - loss: 0.3251 - accuracy: 0.8739 - val_loss: 0.3822 - val_accuracy: 0.8571
Epoch 5/27
37/37 [==============================] - 0s 2ms/step - loss: 0.3127 - accuracy: 0.8807 - val_loss: 0.3818 - val_accuracy: 0.8571
Epoch 6/27
37/37 [==============================] - 0s 2ms/step - loss: 0.3018 - accuracy: 0.8893 - val_loss: 0.3761 - val_accuracy: 0.8571
Epoch 7/27
37/37 [==============================] - 0s 2ms/step - loss: 0.2906 - accuracy: 0.8842 - val_loss: 0.3737 - val_accuracy: 0.8571
Epoch 8/27
37/37 [==

In [ ]:
#predicting the labels
y_pred_encoded = model.predict(X_test)
y_pred = np.argmax(y_pred_encoded, axis=1)

#Calculating the accuracy
acc = accuracy_score(y_test, y_pred)
print('Accuracy', acc)

#Classification report
print(classification_report(y_test, y_pred))

#Print confusion matrix
cm=confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

6/6 [==============================] - 0s 2ms/step
Accuracy 0.7989130434782609
              precision    recall  f1-score   support

           0       0.74      0.81      0.77        77
           1       0.85      0.79      0.82       107

    accuracy                           0.80       184
   macro avg       0.79      0.80      0.80       184
weighted avg       0.80      0.80      0.80       184

Confusion Matrix:
[[62 15]
 [22 85]]


In [ ]:
from sklearn.metrics import precision_score

prec = precision_score(y_test, y_pred)

prec

0.85